In [ ]:
# %%
# Cell 1: Clean up and install dependencies
# Uninstall existing installations to avoid conflicts
!pip uninstall -y tensorflow tensorflow-addons

# Install TensorFlow version 2.13.0
!pip install tensorflow==2.13.0

# Install TensorFlow-Addons version 0.21.0, compatible with TensorFlow 2.13.0
!pip install tensorflow-addons==0.21.0

#Install other modules
!pip install numpy matplotlib

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
# Cell 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, Model
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Cell 3: Define helper functions
def load_images(path, size=(256, 256)):
    """Load and preprocess images from a given path."""
    images = []
    for filename in os.listdir(path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = tf.keras.preprocessing.image.load_img(os.path.join(path, filename), target_size=size)
            img = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img)
    return np.array(images)

def normalize_images(images):
    """Normalize images to [-1, 1]."""
    return (images / 127.5) - 1.0

def denormalize_images(images):
    """Denormalize images to [0, 255]."""
    return ((images + 1.0) * 127.5).astype(np.uint8)

def show_preds(original, transformed, reconstructed):
    """Visualize original, transformed, and reconstructed images."""
    plt.figure(figsize=(12, 4))
    for i, img in enumerate([original, transformed, reconstructed]):
        plt.subplot(1, 3, i + 1)
        plt.imshow((img + 1) / 2)
        plt.axis('off')
    plt.show()

# Cell 4: Define CycleGAN models
def build_generator(image_shape):
    """Build a generator model."""
    inputs = layers.Input(shape=image_shape)
    x = layers.Conv2D(64, (7, 7), padding='same', kernel_initializer='he_normal')(inputs)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.ReLU()(x)

    for _ in range(2):
        x = layers.Conv2D(128, (3, 3), strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = tfa.layers.InstanceNormalization()(x)
        x = layers.ReLU()(x)

    for _ in range(9):
        x = residual_block(x)

    for _ in range(2):
        x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = tfa.layers.InstanceNormalization()(x)
        x = layers.ReLU()(x)

    outputs = layers.Conv2D(3, (7, 7), padding='same', activation='tanh', kernel_initializer='he_normal')(x)
    return Model(inputs, outputs)

def build_discriminator(image_shape):
    """Build a discriminator model."""
    inputs = layers.Input(shape=image_shape)
    x = layers.Conv2D(64, (4, 4), strides=2, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.LeakyReLU(0.2)(x)

    for filters in [128, 256, 512]:
        x = layers.Conv2D(filters, (4, 4), strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = tfa.layers.InstanceNormalization()(x)
        x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(1, (4, 4), padding='same', kernel_initializer='he_normal')(x)
    return Model(inputs, x)

def residual_block(x):
    """Define a residual block."""
    inputs = x
    x = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = tfa.layers.InstanceNormalization()(x)
    return layers.Add()([inputs, x])


In [ ]:


# Cell 5: Load datasets
celeb_path = '/content/drive/MyDrive/Colab Notebooks/dupe'
cartoon_path = '/content/drive/MyDrive/Colab Notebooks/cartoonset10k'

# Load and preprocess images
try:
    dataA = normalize_images(load_images(celeb_path))
    dataB = normalize_images(load_images(cartoon_path))
    print(f"Dataset A shape: {dataA.shape}, Dataset B shape: {dataB.shape}")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")




Dataset A shape: (1924, 256, 256, 3), Dataset B shape: (1030, 256, 256, 3)


In [ ]:
# Cell 6: Training setup
image_shape = (256, 256, 3)
g_model_AtoB = build_generator(image_shape)
g_model_BtoA = build_generator(image_shape)
d_model_A = build_discriminator(image_shape)
d_model_B = build_discriminator(image_shape)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
# Compile discriminators
d_model_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
d_model_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Ensure files is imported
from google.colab import files
from PIL import Image
import io

def upload_and_cartoonize():
    """Allow user to upload an image and apply the transformation."""
    uploaded = files.upload()  # Trigger file upload dialog
    for filename in uploaded.keys():
        # Load the image from the uploaded file
        content = uploaded[filename]
        img = Image.open(io.BytesIO(content)).resize((256, 256))
        img_array = np.array(img)
        img_normalized = normalize_images(np.expand_dims(img_array, axis=0))

        # Perform transformation
        transformed_image = g_model_AtoB.predict(img_normalized)[0]
        cartoonized_image = denormalize_images(transformed_image)

        # Display results
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(img)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("Cartoonized Image")
        plt.imshow(cartoonized_image.astype(np.uint8))
        plt.axis('off')
        plt.show()


# New Section

In [ ]:
# Cell 8: Test the user interface
try:
    upload_and_cartoonize()
except Exception as e:
    print(f"Error during cartoonization: {e}")


Error during cartoonization: name 'upload_and_cartoonize' is not defined
